<a href="https://colab.research.google.com/github/Yapping72/ICT3102-e-mc2-assignment-1/blob/main/ICT3102_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `Project Dependencies`

In [1]:
!pip install transformers
!pip install sentencepiece
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00


# `Project Code`

In [18]:
from transformers import AutoTokenizer
from datasets import load_dataset

# byte_pair_tokenization = ["openai-gpt", "gpt2", "NousResearch/Llama-2-13b-hf"]
# unigram_tokenization = ["google/bigbird-roberta-base", "facebook/mbart-large-50-many-to-many-mmt" , "albert-base-v2" , "xlnet-base-cased"]
# wordpiece_tokenization = ['distilbert-base-uncased','google/mobilebert-uncased','funnel-transformer/small-base','sentence-transformers/all-mpnet-base-v2']
# sentencepiece_tokenization = ["google/flan-t5-base"]

corpus = ["I have a new GPU!", "I wonder how fast the model will train on this.", "Car park there"]


dataset = load_dataset("HuggingFaceH4/self-instruct-seed")
# corpus = dataset['train']['instruction']

def initialize_model(model_name:str):
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  return tokenizer

def time_model(tokenizer_object, corpus):
    """Calculate timing for encode_plus"""
    # Capture the timeit result
    total_time = 0
    for text in corpus:
        # Capture the timeit result for each text
        timeit_result = %timeit -n 0 -r 1 -o tokenizer.encode_plus(text)
        total_time += timeit_result.average
    # Return the average time in milliseconds
    return total_time * 1e3

def time_model_batch(tokenizer, corpus):
    """Calculate timing for batch_encode_plus"""
    # Capture the timeit result
    timeit_result = %timeit -n 0 -r 1 -o tokenizer.batch_encode_plus(corpus)
    # Return the average time in milliseconds
    return timeit_result.average * 1e3

def analyse_encode_plus(tokenizers: list, corpus: list) -> dict:
    results = {}
    results['method'] = "Unbatched"
    for hugging_face_tokenizer in tokenizers:
        try:
          tokenizer = initialize_model(hugging_face_tokenizer)
          formattedCorpus = tokenizer(corpus, truncation=True)
          # print(formattedCorpus)
          average_time = time_model(tokenizer, formattedCorpus)

          # Extract tokenizer name or path for dictionary key
          tokenizer_name = tokenizer.name_or_path
          results[tokenizer_name] = average_time
        except Exception as e:
          print(f"Error occured for {hugging_face_tokenizer}: {e}")
          continue

    return results

def analyse_batch(tokenizers: list, corpus: list) -> dict:
    results = {}
    results['method'] = "Batched"
    for hugging_face_tokenizer in tokenizers:
        try:
          tokenizer = initialize_model(hugging_face_tokenizer)
          average_time = time_model_batch(tokenizer, corpus)

          # Extract tokenizer name or path for dictionary key
          tokenizer_name = tokenizer.name_or_path
          results[tokenizer_name] = average_time
        except Exception as e:
          print(f"Error occured for {hugging_face_tokenizer}: {e}")
          continue

    return results

In [19]:
byte_pair = ["openai-gpt", "gpt2", "NousResearch/Llama-2-13b-hf"]
byte_pair_timing_unbatched = analyse_encode_plus(byte_pair, corpus)
byte_pair_timing_batched = analyse_batch(byte_pair,corpus)

69.8 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 10000 loops each)
63.9 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 10000 loops each)
34.9 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 10000 loops each)
38 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 10000 loops each)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


36.4 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 10000 loops each)
67.4 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 10000 loops each)
197 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1000 loops each)
262 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1000 loops each)
224 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1000 loops each)


In [11]:
# Get response time for word_piece models
word_piece =['distilbert-base-uncased','google/mobilebert-uncased','funnel-transformer/small-base','sentence-transformers/all-mpnet-base-v2']
word_piece_timing_unbatched = analyse_encode_plus(word_piece, corpus)
word_piece_timing_batched = analyse_batch(word_piece,corpus)

137 µs ± 33.2 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
101 µs ± 10.6 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
237 µs ± 49.3 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
176 µs ± 17.8 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
189 µs ± 10.7 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
205 µs ± 18.6 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
193 µs ± 12 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
179 µs ± 11 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [12]:
sentence_piece = ["google/flan-t5-base"]
sentence_piece_timing_unbatched = analyse_encode_plus(sentence_piece, corpus)
sentence_piece_timing_batched = analyse_batch(sentence_piece,corpus)

88.8 µs ± 26.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
156 µs ± 11 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [13]:
unigram = ["google/bigbird-roberta-base", "facebook/mbart-large-50-many-to-many-mmt" , "albert-base-v2" , "xlnet-base-cased"]
unigram_timing_unbatched = analyse_encode_plus(unigram, corpus)
unigram_timing_batched = analyse_batch(unigram,corpus)

79.3 µs ± 16.7 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
74.2 µs ± 1.01 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
132 µs ± 38.7 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
105 µs ± 26.9 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
143 µs ± 2.98 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
156 µs ± 4.26 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
200 µs ± 11.8 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
188 µs ± 12.6 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [22]:
print(byte_pair_timing_unbatched)
print(byte_pair_timing_batched)
print(word_piece_timing_unbatched)
print(word_piece_timing_batched)
print(sentence_piece_timing_unbatched)
print(sentence_piece_timing_batched)
print(unigram_timing_unbatched)
print(unigram_timing_batched)

{'method': 'Unbatched', 'openai-gpt': 0.13366224120002243, 'gpt2': 0.07297341069997855, 'NousResearch/Llama-2-13b-hf': 0.10379259730000286}
{'method': 'Batched', 'openai-gpt': 0.19661517999998068, 'gpt2': 0.26245640100000855, 'NousResearch/Llama-2-13b-hf': 0.22394624000003205}
